**Table of contents**    
1. Setting    
2. Data Load    
2.1. 상품정보    
2.2. 리뷰정보    
3. Modeling    
3.1. predict with pretrained model    
3.2. fine tuning    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=false
	flat=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

<br>

# 1. Setting

In [2]:
# import os
# os.chdir(os.getcwd() + '/../..')
# os.getcwd()

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/Storage/Github/hyuckjinkim')

Mounted at /content/drive


In [3]:
!pip install -q swifter
!pip install -q transformers datasets accelerate kiwipiepy evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16507 sha256=fa7440aa78d176593b1fcaccb3f5e1360158590d041b56f3e40226c0a60f134a
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
Successfully built swifter
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [4]:
import glob
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
from joblib import Parallel, delayed
import gc
import json
import swifter

tqdm.pandas()
pd.set_option('display.max_columns', None)
gc.collect()

60

In [106]:
ROOT_DIR = 'crawling/naver_shopping_review'
SAVE_DIR = os.path.join(ROOT_DIR, '.result/')
MC_DIR = os.path.join(ROOT_DIR, '.model_checkpoints/')

<br>

# 2. Data Load

<br>

## 2.1. 상품정보

In [6]:
products_path = '20240701_선크림_5_100/product_page5.parquet'
products_info = pd.read_parquet(os.path.join(SAVE_DIR, products_path))

# 폴더경로로부터 수집일자, 수집키워드, 상품수집페이지, 최대리뷰수집페이지를 가져온다
collected_date, keyword, page, max_review_page = products_path.split('/')[0].split('_')
print(f'{collected_date = }\n{keyword = }\n{page = }\n{max_review_page = }')

# 네이버쇼핑 상품정보 전처리
# (1) 스마트스토어가 아닌 상품은 리뷰를 가져올수없으므로 제거
products_info['is_smartstore'] = products_info['mallProductUrl'].str.contains('https://smartstore.naver.com/main/products').astype(int)
products_info = products_info[products_info['is_smartstore']==1].reset_index(drop=True)

# (2) 리뷰가 0인 상품들 제거
products_info = products_info[products_info['reviewCount']!='0'].reset_index(drop=True)

products_info.head(2)

collected_date = '20240701'
keyword = '선크림'
page = '5'
max_review_page = '100'


,keyword,collection,purchaseConditionInfos,rank,id,parentId,hasLowestCardPrice,hasAddInFee,scoreInfo,category1Id,category2Id,category3Id,category4Id,category1Name,category2Name,category3Name,category4Name,category4NameOrg,categoryLevel,openDate,maker,makerNo,brand,brandNo,series,seriesNo,attributeValueSeqs,attributeValue,characterValueSeqs,characterValue,productName,productTitle,productTitleOrg,descriptionOrder,searchKeyword,officialCertifiedLowPrice,lowPrice,mobileLowPrice,price,mobilePrice,dlvryPrice,dlvryLowPrice,priceUnit,lowestCardPrice,lowestCardName,reviewCount,checkOutReviewCount,reviewCountSum,imageUrl,additionalImageCount,mallCount,mallNo,mallId,mallName,mallNameOrg,mallProductId,originalMallProductId,mallProductUrl,mallProdMblUrl,mallPcUrl,mallSectionNo,isBrandStore,isNaverPay,isMblNaverPay,nPayPcType,nPayMblType,naverPayAccumRto,atmtTag,manuTag,preferTag,purchaseCnt,smryReview,isHotDeal,isLgtModelMat,lnchYm,keepCnt,prchCondInfo,exchangeRateInfo,mpTp,overseaTp,saleTp,prodTp,wdTp,wdNm,comNm,rentalCont,imgSz,isChnlPnt,chnlSeq,mallInfo,isMisImg,dlvryCd,dlvryCont,dlvryLowPriceByMallNo,fastdlvry,fastdlvryCont,lgstDlvryCont,rmid,stdPrchOptSeqs,stdPrchOptNames,stdPrchOptValSeqs,stdPrchOptValNames,stdPrchCondInfo,stdPrchOptCount,parentCatalogId,stdPrchOptVal,stdPrchOptHit,rankReviewCount,stdCatlogMatchType,stdGroupId,isAdult,isAdultExpsRstct,videoId,hasVideo,gdid,deliveryFeeContent,diffDeliveryFeeContent,eventContent,hasEventContent,couponContent,hasCouponContent,cardContent,hasCardContent,buyPointContent,hasBuyPointContent,preorderCont,imgColorCd,imgSgnt,imgVersion,lowPriceByMallNo,shopNNo,dummy,lowMallList,mallInfoCache,channelInfoCache,crUrl,crUrlMore,standardPurchaseCondtionInfos,adCntsSeq,sellerDeliveryInfo,is_smartstore
0,선크림,product,"[{'seq': '', 'condition': '', 'count': '', 'lo...",1,82533701144,82533701144,0,0,4.78,50000002,50000191,50000445,,화장품/미용,선케어,선크림,,,3,20200626183838,,0,닥터바이오,257460,,0,10011958 10011958 10011730 10011730 10011730 1...,사용부위|사용부위|피부타입|피부타입|피부타입|종류|자외선차단지수|PA지수|주요제품특...,M10458898 M10666274 M10416374 M10416375 M10416...,페이스용|바디용|모든피부용|건성|민감성|유기자차|50|PA++++|촉촉함(수분공급)...,닥터바이오 온가족용 에코 선크림(100g) 눈시림 백탁 현상 없는 건성 지성 저자극...,닥터바이오 온가족용 에코 선크림(100g) 눈시림 백탁 현상 없는 건성 지성 저자극...,닥터바이오 온가족용 에코 선크림(100g) 눈시림 백탁 현상 없는 건성 지성 저자극...,RV,,^,18400,18400,18400,18400,18400,18400,KRW,None,,4498,0,4498,https://shopping-phinf.pstatic.net/main_825337...,5,0,721498,ncp_1nn7bn_01,닥터바이오 더마 코스메틱,닥터바이오 더마 코스메틱,4989180477,4972187149,https://smartstore.naver.com/main/products/498...,https://m.smartstore.naver.com/main/products/4...,https://smartstore.naver.com/hgcosstore,10,1,1,1,2,2,0,,"순한선크림,신생아,여름,국민템,순한썬크림,뽀송썬로션,순한제품,아기피부를촉촉하게,오늘...",32767.5\t0f0,3678,,0,0,,936,|||||||,,1,0,0,1,0,,,,1000x1000,None,100200351,1,0,771,v4!#!오늘출발^15:00^택배^CJ대한통운^^^0^0^^1.5^토요일|일요일/^...,,"t00,t01,t02,t03,t04,t05,t06,t07,t08,t09,t10,t1...",,,,,,,,^^^^^^^,0,,,0,4499,0,82533701144,0,0,,0,00000009_001337645618,0,,,0,2100원^^^1500,1,,0,^184^184^^^0^0,1,,0,50209594052299732601025280,15,,721498,,None,"{'seq': '721498', 'prodCnt': '176', 'name': '닥...",{'talkAccountId': 'wcbqvt'},https://cr.shopping.naver.com/adcr.nhn?x=nePSD...,https://cr.shopping.naver.com/adcr.nhn?x=GGV%2...,None,None,None,1
1,선크림,product,"[{'seq': '', 'condition': '', 'count': '', 'lo...",2,85970708499,85970708499,0,0,4.81,50000002,50000191,50000445,,화장품/미용,선케어,선크림,,,3,20230419181203,비오레,241766,카오,100321,,0,10011958 10011730 10028894 10011722 10019292 1...,사용부위|피부타입|종류|자외선차단지수|주요제품특징|주요제품특징|주요제품특징,M10458898 M10416374 M11030325 M10415567 M10818...,페이스용|모든피부용|혼합자차(유기+무기)|50|촉촉함(수분공급)|부드러운 발림|백탁...,비오레 UV 일본 썬크림 아쿠아리치 워터리 에센스 선크림 70g,비오레 UV 일본 썬크림 아쿠아리치 워터리 에센스 선크림 70g,비오레 UV 일본 썬크림 아쿠아리치 워터리 에센스 선크림 70g,RV,,^,7550,7550,7550,7550,10550,10550,KRW,None,,1394,0,1394,https://shopping-phinf.pstatic.net/main_859707...,9,0,3458038,ncp_1o632j_01,일본직구 엔핍,일본직구 엔핍,8426208176,8386016777,https://smartstore.naver.com/main/products/842...,https://m.smartstore.naver.com/main/products/8...,https://sma

<br>

## 2.2. 리뷰정보

In [ ]:
# dirs = glob.glob(os.path.join(SAVE_DIR,'*'))
# dirs = [dir.replace('\\','/').split('/')[-1] for dir in dirs]

# # 3m
# df = []
# for dirs_iter in range(len(dirs)):
#     # 폴더명으로부터 수집키워드, 수집일자 가져오기
#     collected_date, keyword, page, max_review_page = dirs[dirs_iter].split('_')
#     desc = f'[{dirs_iter+1}/{len(dirs)}] {collected_date=}, {keyword=}, {page=}, {max_review_page=}'

#     # 저장된 리뷰경로들 가져오기
#     review_paths = glob.glob(os.path.join(SAVE_DIR,dirs[dirs_iter],'review_product*.parquet'))

#     # 리뷰 가져오기
#     n_jobs = os.cpu_count()
#     d = Parallel(n_jobs=n_jobs, prefer='threads')(
#         delayed(pd.read_parquet)(path) for path in tqdm(review_paths, desc=desc)
#     )
#     d = pd.concat(d,axis=0)

#     # 수집키워드, 수집일자 넣기
#     d.insert(0, 'keyword', keyword)
#     d.insert(1, 'collected_date', collected_date)

#     # append
#     df.append(d)

# df = pd.concat(df,axis=0)
# df = df.sort_values(['keyword','collected_date','product_ranking','review_ranking'])
# df = df.reset_index(drop=True)

In [ ]:
# df.to_parquet(os.path.join(SAVE_DIR, 'review_df.parquet'))
df = pd.read_parquet(os.path.join(SAVE_DIR, 'review_df.parquet'))

In [ ]:
print(df.shape)
df.head(2)

In [ ]:
# reviewType : NORMAL, AFTER_USE
# reviewServiceType : SELLBLOG
# reviewContentClassType : PHOTO, TEXT, VIDEO
# reviewScore : 1,2,3,4,5
# *reviewContent : 리뷰 본문
# contentsStatusType : NORMAL
# createDate
# freeTrial : False, nan
# repurchase : False, True, nan
# reviewRankingScore
# writerId
# maskedWriterId
# writerIdNo
# writerMemberNo
# writerProfileImageUrl
# storeType : STOREFARM
# storeNo
# checkoutMerchantId
# checkoutMerchantNo
# orderNo
# productOrderNo
# productNo
# productName
# productUrl
# largeCategorizeCategoryId
# middleCategorizeCategoryId
# smallCategorizeCategoryId
# productOptionContentNoDisplay : False
# knowledgeShoppingMallProductId
# originProductNo
# reviewAttaches : 리뷰 첨부파일
# reviewCommentIds : 댓글로 보임
# reviewComments
# reviewEvaluationValueIds
# reviewUserInfoValues
# *reviewTopics
# eventTitle
# profileImageSourceType : DEFAULT, NAVER_PROFILE
# repThumbnailAttach : 썸네일 첨부파일
# repThumbnailTagNameDescription
# helpCount
# parentReviewId
# bestReview : nan, True, False
# bestReviewSelectDate
# benefitPaymentDate
# modifyDate
# reviewInspectionPolicyReason : nan, 관련없는 이미지
# productOptionContent : 상품 옵션
# standardPurchaseConditionText

In [ ]:
# (1) unique 컬럼 제거
nunique_info = df.nunique()
unique_info = df.apply(lambda x: x.unique())
unique_cols = nunique_info[nunique_info==1].index.tolist()
display(unique_info[unique_cols])

df.drop(unique_cols, axis=1, inplace=True)

In [ ]:
# df.groupby('keyword')['largeCategorizeCategoryId'].value_counts()

In [ ]:
# (2) 사용하지 않는 컬럼들 제거
id_cols = ['id','writerId','maskedWriterId','storeNo','productOrderNo','productNo',
           'largeCategorizeCategoryId','middleCategorizeCategoryId','smallCategorizeCategoryId',
           'knowledgeShoppingMallProductId','originProductNo','reviewEvaluationValueIds','reviewCommentIds']
url_cols = ['writerProfileImageUrl','productUrl']
df.drop(id_cols+url_cols, axis=1, inplace=True)

In [ ]:
# 병렬 처리 함수
def parallel_apply(df, func, **kwargs):
    pbar = tqdm(df.iterrows(), total=len(df), position=0, leave=False)
    results = Parallel(n_jobs=-1, prefer='threads')(
        delayed(func)(row, **kwargs) for _, row in pbar
    )
    return results

# 토픽에 해당하는 문장을 추출
def extract_topic_sentences(review_content, review_topics):
    review_topics = str(review_topics)
    if review_topics.upper() in ['NAN','NONE']:
        return None

    # 토픽정보 string -> dict
    # review_topics = eval(review_topics)
    review_topics = json.loads(review_topics.replace("\'", "\""))

    # patternStartNo, patternEndNo를 통해 토픽에 해당하는 리뷰문장을 가져오기
    topic_info = [list(topic.values()) for topic in review_topics]
    topic_text = [review_content[info[2]:info[3]+1] for info in topic_info]

    # 토픽영문명, 토픽한글명, 토픽에 해당하는 리뷰문장
    result = np.concatenate([
        np.array(topic_info)[:,:2].tolist(), # 토픽영문명, 토픽한글명
        np.expand_dims(topic_text,axis=1)    # 토픽에 해당하는 리뷰 문장
    ], axis=1).tolist()

    return result

In [ ]:
# parallel_apply(df, lambda row: extract_topic_sentences(row['reviewContent'], row['reviewTopics']))
df['topic_sentences'] = df[['reviewContent','reviewTopics']]\
    .swifter.set_npartitions(8)\
    .apply(lambda x: extract_topic_sentences(x['reviewContent'],x['reviewTopics']), axis=1)

In [ ]:
# 토픽문장 기준으로 melting
# ex) 리뷰 1번에 토픽이 3개 있는 경우, 행이 3개로 늘어남
def melt_by_topic_sentence(df, iter):
    # 해당 iter의 토픽 문장
    topic_sentences = df['topic_sentences'].values[iter]

    # 리뷰에 토픽이 존재하지 않으면, 해당 리뷰는 제거됨
    if topic_sentences is None:
        return None

    d = pd.DataFrame(topic_sentences,
                     columns=['topic_eng','topic_kor','topic_sentence'],
                     index=[df.index[iter]]*len(topic_sentences))
    return d

In [ ]:
melted_topic_sentences_df = pd.concat([melt_by_topic_sentence(df,iter) for iter in trange(len(df))])
melted_df = pd.merge(
    df.drop('topic_sentences',axis=1).reset_index(),
    melted_topic_sentences_df.reset_index(),
    how='left',
    on='index',
)
melted_df.to_parquet(os.path.join(SAVE_DIR,'melted_review_df.parquet'))

In [ ]:
melted_df = pd.read_parquet(os.path.join(SAVE_DIR,'melted_review_df.parquet'))

In [ ]:
print(melted_df.shape)
melted_df.head(2)

In [ ]:
t = melted_df[melted_df['index']==0].copy()
print(t['reviewContent'].values[0],'\n\n')
t[['topic_kor','topic_sentence']].values

<br>

# 3. Modeling

<br>

## 3.1. predict with pretrained model

In [ ]:
import spacy
from transformers import pipeline

# 스페이시 모델 로드
nlp = spacy.load("en_core_web_sm")

# 트랜스포머 파이프라인을 사용하여 감정 분석 모델 로드
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [ ]:
# 리뷰 텍스트
review_text = """
평소에 얼굴에 기미가 잘 끼고 피부가 워낙 얕은 편이라 자외선에 조금만 노출되어도 잡티가 많이 생기는 민감하고 아주 예민한 피부타입 입니다~~.
1년 365일 단 하루도 썬크림을 안바르는 날이 없고 또
2시간에 한번씩 썬스틱을 엄청 바르는 편입니다.
기초 화장품 바르는 것 이외에 썬스틱을 바르고 그 외에는 다른 메이크업 베이스나 쿠션같은건 안하는 타입입니다. 얼굴에 두껍게 뭔가 바르는걸 답답해 하는 타입이라 썬스틱이 가장 마지막에 바르는 화장품 입니다.
예전에는 썬스틱이 판매되지 않을 때에는 액체형 썬크림을 바르는게 손에도 뭍고 다 펴바르는 데에도 흡수시키고 하렴 시간도 걸리고 불편하기도 했는데 썬스틱이 출시된 후에는 신세계가 열린 기분이 들었습니다~^^.
썬크림을 1년 내내 부지런히 매시간 자주 바르는 저한테는 썬스틱은 아주 효자 상품 이랍니다 ㅎㅎ.
썬스틱을 자주 발라 주고 집에서도 조명때문에 예민한
피부가 상할까바 늘 부지런히 발라서 썬스틱을 한달에
1통 정도는 쓰는 편입니다.
네이버 쇼핑으로 가격도 저렴하고 좋은제품 가성비좋게 구매해서 넘 좋아요~^^. 그리고 AHC썬스틱은 몇년째 계속 구매해서 쓰는 제품인데 부드럽게 잘 펴발라지고 ,색도 튀지않게 피부에 잘 맞고, 향도좋고 여러모로
쓰기편하고 만족도가 아주 높은 썬스틱 입니다~^^.
50대가 다 된 나이에도 나이 모르는 외부사람들은 30대라고 할 정도로 피부가 주름없이 좋은 편입니다.
비결은 평상시에 썬크림을 부지런히 발라줘서 주름도 많이 방지 되는데 있는 것 같습니다~^^. 피부화장은 안해도 썬크림이나 썬스틱은 부지런히 발라주는게 내 피부를 지키는 비결이라 생각합니다~^^. 앞으로도 꾸준히 썬스틱 재구매 할 생각입니다. 좋은제품 앞으로도 계속 만들어 주시면 감사드립니다 ~.
"""

# 주제 목록
topics = ["만족도", "가격", "사용감", "효과"]

# 리뷰 텍스트를 문장으로 분리
doc = nlp(review_text)
sentences = [sent.text.strip() for sent in doc.sents]

# 주제별 문장 분류
results = []
for sentence in sentences:
    result = classifier(sentence, topics)
    topic = result["labels"][0]
    if result["scores"][0] > 0.5:  # 임계값 설정 (0.5 이상일 때만 주제로 분류)
        results.append([topic, sentence])

# 결과 출력
import numpy as np
results_array = np.array(results)
print(results_array)

In [ ]:
import torch
from torch.nn.functional import softmax
from transformers import BartForSequenceClassification, PreTrainedTokenizerFast
from kiwipiepy import Kiwi
import numpy as np

# KoBART 모델 로드
model_name = 'gogamza/kobart-base-v1'
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
model = BartForSequenceClassification.from_pretrained(model_name, num_labels=4)

In [ ]:
model

In [ ]:
# 토픽(주제) 설정
topics = ["만족도", "가격", "사용감", "효과"]

# 리뷰 텍스트
review_text = """
평소에 얼굴에 기미가 잘 끼고 피부가 워낙 얕은 편이라 자외선에 조금만 노출되어도 잡티가 많이 생기는 민감하고 아주 예민한 피부타입 입니다~~.
1년 365일 단 하루도 썬크림을 안바르는 날이 없고 또
2시간에 한번씩 썬스틱을 엄청 바르는 편입니다.
기초 화장품 바르는 것 이외에 썬스틱을 바르고 그 외에는 다른 메이크업 베이스나 쿠션같은건 안하는 타입입니다. 얼굴에 두껍게 뭔가 바르는걸 답답해 하는 타입이라 썬스틱이 가장 마지막에 바르는 화장품 입니다.
예전에는 썬스틱이 판매되지 않을 때에는 액체형 썬크림을 바르는게 손에도 뭍고 다 펴바르는 데에도 흡수시키고 하렴 시간도 걸리고 불편하기도 했는데 썬스틱이 출시된 후에는 신세계가 열린 기분이 들었습니다~^^.
썬크림을 1년 내내 부지런히 매시간 자주 바르는 저한테는 썬스틱은 아주 효자 상품 이랍니다 ㅎㅎ.
썬스틱을 자주 발라 주고 집에서도 조명때문에 예민한
피부가 상할까바 늘 부지런히 발라서 썬스틱을 한달에
1통 정도는 쓰는 편입니다.
네이버 쇼핑으로 가격도 저렴하고 좋은제품 가성비좋게 구매해서 넘 좋아요~^^. 그리고 AHC썬스틱은 몇년째 계속 구매해서 쓰는 제품인데 부드럽게 잘 펴발라지고 ,색도 튀지않게 피부에 잘 맞고, 향도좋고 여러모로
쓰기편하고 만족도가 아주 높은 썬스틱 입니다~^^.
50대가 다 된 나이에도 나이 모르는 외부사람들은 30대라고 할 정도로 피부가 주름없이 좋은 편입니다.
비결은 평상시에 썬크림을 부지런히 발라줘서 주름도 많이 방지 되는데 있는 것 같습니다~^^. 피부화장은 안해도 썬크림이나 썬스틱은 부지런히 발라주는게 내 피부를 지키는 비결이라 생각합니다~^^. 앞으로도 꾸준히 썬스틱 재구매 할 생각입니다. 좋은제품 앞으로도 계속 만들어 주시면 감사드립니다 ~.
"""

# 텍스트를 문장으로 분리
kiwi = Kiwi()
sentences = [sentence.text for sentence in kiwi.split_into_sents(review_text)]

In [ ]:
# 주제별 문장 분류 함수
def classify_sentences(sentences, model, tokenizer, topics):
    results = []
    for sentence in sentences:
        sentence = sentence.strip() + " </s>"
        inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
        inputs.pop('token_type_ids', None)
        outputs = model(**inputs)
        logits = outputs.logits
        prob = softmax(logits, dim=-1).detach().numpy()
        topic_idx = torch.argmax(logits, dim=1).item()
        topic = topics[topic_idx]
        results.append([topic, sentence, prob.max()])
    return results

# 주제별 문장 분류
results = classify_sentences(sentences, model, tokenizer, topics)

# 결과 출력
results_array = np.array(results)
print(results_array)

In [ ]:
np.array([['만족도', '얼굴에 두껍게 뭔가 바르는걸 답답해 하는 타입이라 썬스틱이 가장 마지막에 바르는 화장품 입니다'],
       ['가격', '네이버 쇼핑으로 가격도 저렴하고'],
       ['만족도', '좋은제품 가성비좋게 구매해서 넘 좋아요'],
       ['만족도', '색도 튀지않게 피부에 잘 맞고, 향도좋고'],
       ['만족도', '좋은제품 앞으로도 계속 만들어 주시면 감사드립니다']], dtype=object)

<br>

## 3.2. fine tuning

In [95]:
import pandas as pd
import torch
import torch.nn as nn
from transformers import BartForSequenceClassification, BartTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import Dataset, DatasetDict, load_metric
import evaluate
from sklearn.metrics import accuracy_score, f1_score

In [96]:
import accelerate
accelerate.__version__

'0.32.1'

In [97]:
# 토크나이징 함수
def tokenize(batch):
    return tokenizer(batch['topic_sentence'], padding=True, truncation=True)

def compute_metrics(eval_pred):
    # second element seems to be an output of a layer of my BART model
    # (https://stackoverflow.com/questions/76349622/training-a-bartforsequenceclassification-returns-data-with-ununiform-dimentsions)
    logits = eval_pred.predictions[0]
    labels = eval_pred.label_ids
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")
    return {"accuracy": acc, "f1": f1}

In [98]:
melted_df = pd.read_parquet(os.path.join(SAVE_DIR, 'melted_review_df.parquet'))

In [155]:
df = melted_df.copy()

df = df[(df['keyword']=='유산균') & (df['topic_kor'].notnull())].copy()
df = df[['index','keyword','reviewContent','topic_sentence','topic_kor']]

vc = df['topic_kor'].value_counts()
selected_topic = vc[vc>100].index.tolist()
df = df[df['topic_kor'].isin(selected_topic)]

# df = df.sample(len(df) // 100, random_state=0)
# # df = df.sample(100)
# df.reset_index(inplace=True)

print(len(df), df.topic_kor.nunique())

197247 8


In [156]:
df['topic_kor'].value_counts()

topic_kor
만족도     144739
배송       29002
맛        18439
유통기한      3396
건강         678
품질         639
용량         178
배달         176
Name: count, dtype: int64

In [139]:
# 라벨을 숫자로 변환
label2num = {label: idx for idx, label in enumerate(df['topic_kor'].unique())}
num2label = {idx: label for idx, label in enumerate(df['topic_kor'].unique())}
df['label'] = df['topic_kor'].map(label2num)

# Dataset으로 변환
dataset = Dataset.from_pandas(df[['topic_sentence','label']])

# 데이터셋 분리
dataset = dataset.train_test_split(test_size=0.2)
dataset = DatasetDict({"train": dataset["train"], "test": dataset["test"]})

In [101]:
# 토크나이저 로드
model_name = 'gogamza/kobart-base-v1'
tokenizer = BartTokenizer.from_pretrained(model_name)

# 데이터셋 토크나이징
dataset = dataset.map(tokenize, batched=True)
dataset.set_format('torch', columns=['input_ids','attention_mask','label'])

# 데이터 콜레이터 설정
data_collator = DataCollatorWithPadding(tokenizer)

# 모델 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BartForSequenceClassification.from_pretrained(model_name, num_labels=len(label2num))
model.to(device)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'BartTokenizer'.


Map:   0%|          | 0/1579 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/395 [00:00<?, ? examples/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Some weights of BartForSequenceClassification were not initialized from the model checkpoint at gogamza/kobart-base-v1 and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BartForSequenceClassification(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): Lay

In [102]:
class config:
    learning_rate = 2e-5
    batch_size = 8
    epochs = 3
    weight_decay = 0.01

In [103]:
# 클래스 가중치 계산
def compute_class_weights(classes, y):
    class_counts = np.bincount(classes)
    total_samples = len(y)
    num_classes = len(class_counts)
    class_weights = total_samples / (num_classes * class_counts)
    return class_weights

# 커스텀 Trainer 클래스
class WeightedLossTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_function = torch.nn.CrossEntropyLoss(weight=self.class_weights).to(model.device)
        loss = loss_function(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [108]:
# class weight 계산
unique_classes, class_indices = np.unique(df['label'], return_inverse=True)
class_weights = compute_class_weights(class_indices, df['label'])
class_weights = torch.tensor(class_weights, dtype=torch.float)

# 훈련 인자 설정
training_args = TrainingArguments(
    output_dir=MC_DIR,
    evaluation_strategy="epoch",
    learning_rate=config.learning_rate,
    per_device_train_batch_size=config.batch_size,
    per_device_eval_batch_size=config.batch_size,
    num_train_epochs=config.epochs,
    weight_decay=config.weight_decay,
)

# 트레이너 설정
trainer = WeightedLossTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    class_weights=class_weights,
)

# 모델 훈련
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,1.685867,0.860759,0.850498
2,No log,1.152104,0.873418,0.873151
3,0.800400,1.131796,0.891139,0.889391


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 1}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 1}


TrainOutput(global_step=594, training_loss=0.7573278857401324, metrics={'train_runtime': 148.0206, 'train_samples_per_second': 32.002, 'train_steps_per_second': 4.013, 'total_flos': 289438355484648.0, 'train_loss': 0.7573278857401324, 'epoch': 3.0})

In [129]:
predictions = trainer.predict(dataset["test"])
logits = predictions.predictions[0]
labels = predictions.label_ids
predictions = torch.argmax(torch.tensor(logits), dim=-1)
acc = accuracy_score(labels, predictions)
f1 = f1_score(labels, predictions, average="weighted")

In [133]:
acc, f1

(0.8911392405063291, 0.88939087186882)

In [149]:
pred = [num2label[p] for p in predictions.numpy()]
actual = [num2label[l] for l in labels]

In [150]:
pd.crosstab(actual, pred)

col_0,만족도,맛,배송,유통기한
row_0,,,,
건강,1,0,0,0
만족도,260,6,17,2
맛,6,35,1,0
배달,0,0,1,0
배송,6,0,52,1
용량,0,0,0,1
유통기한,0,0,0,5
품질,1,0,0,0
